In [241]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd
import re

import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [242]:
import json
import warnings

import numpy as np
import pandas as pd

from category_encoders import OrdinalEncoder
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform
from sklearn.metrics import f1_score, accuracy_score

import matplotlib.pyplot as plt

In [2]:
df_main = pd.read_csv('./driver_policy.csv')
vehicles_data = pd.read_csv('./vehicles.csv')

In [4]:
vehicles_data = vehicles_data.drop(['Unnamed: 0', 'ownership_type', 'color', 'make_model'], axis='columns')

In [6]:
new = pd.DataFrame(vehicles_data.groupby('policy_id')['car_no'].max())

In [7]:
new = new.reset_index()

In [9]:
list_sum_age = []
for i in new['policy_id']:
    
    list_sum_age.append(vehicles_data[vehicles_data['policy_id']==i]['age'].sum())

In [11]:
new['sum_age'] = list_sum_age

In [13]:
df_merged = pd.merge(df_main, new)

In [14]:
df_merged = df_merged.drop(['Unnamed: 0', 'X', 'discount', 'Home_policy_ind', 'zip', 'state_id', 'county_name'
                           , 'primary_parking'], axis='columns')

In [15]:
df_merged = df_merged.drop(['Prior_carrier_grp', 'Cov_package_type'], axis = 'columns')

In [16]:
df_merged = df_merged.drop(['Quote_dt', 'Agent_cd', 'CAT_zone'], axis = 'columns')

In [402]:
df_merged

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
0,"$5,153",613.00,policy_87209,2,1,2,0,3,0.00,Train,99,148.00,4,21.00
1,"$9,870",576.00,policy_89288,1,1,1,2,4,NaN,Test,56,62.00,4,11.00
2,"$3,090",631.00,policy_91413,2,1,2,1,4,0.00,Train,98,140.00,4,28.00
3,"$2,860",573.00,policy_23460,1,2,3,0,5,NaN,Test,30,90.00,2,11.00
4,"$14,917",602.00,policy_71845,2,0,1,2,3,0.00,Train,113,163.00,4,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49157,"$4,042",590.00,policy_28284,1,2,2,0,4,0.00,Train,52,90.00,4,20.00
49158,"$5,743",688.00,policy_67016,2,2,2,0,4,0.00,Train,138,147.00,4,25.00
49159,"$2,230",637.00,policy_30163,1,1,3,1,5,0.00,Train,19,39.00,2,14.00
49160,"$8,428",581.00,policy_63982,2,2,2,0,4,1.00,Train,106,156.00,5,24.00


In [19]:
df_train = df_merged[df_merged['split']=='Train']

In [374]:
df_train 

,credit_score,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,driver_age,n_safty_rating,car_no,sum_age,quoted_amt
0,613.00,2,1,2,0,3,0.00,99,148.00,4,21.00,5153
1,631.00,2,1,2,1,4,0.00,98,140.00,4,28.00,3090
2,602.00,2,0,1,2,3,0.00,113,163.00,4,18.00,14917
3,704.00,1,2,2,0,4,0.00,18,66.00,5,30.00,4620
4,611.00,4,2,2,2,6,0.00,93,297.00,3,21.00,11470
...,...,...,...,...,...,...,...,...,...,...,...,...
36494,590.00,1,2,2,0,4,0.00,52,90.00,4,20.00,4042
36495,688.00,2,2,2,0,4,0.00,138,147.00,4,25.00,5743
36496,637.00,1,1,3,1,5,0.00,19,39.00,2,14.00,2230
36497,581.00,2,2,2,0,4,1.00,106,156.00,5,24.00,8428


In [487]:
df_test = df_merged[df_merged['split']=='Test']

In [488]:
df_test

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
1,"$9,870",576.00,policy_89288,1,1,1,2,4,NaN,Test,56,62.00,4,11.00
3,"$2,860",573.00,policy_23460,1,2,3,0,5,NaN,Test,30,90.00,2,11.00
7,"$2,980",679.00,policy_43809,4,0,2,0,2,NaN,Test,91,238.00,2,6.00
8,"$1,945",773.00,policy_4590,3,1,3,2,6,NaN,Test,144,194.00,4,9.00
9,"$5,829",680.00,policy_65525,1,2,3,0,5,NaN,Test,55,72.00,5,21.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49137,"$2,922",727.00,policy_8331,1,3,3,0,6,NaN,Test,43,28.00,4,17.00
49144,"$5,437",658.00,policy_4090,5,1,2,0,3,NaN,Test,133,318.00,4,19.00
49146,"$3,287",586.00,policy_39956,1,2,2,0,4,NaN,Test,18,51.00,1,6.00
49150,"$3,567",693.00,policy_68766,3,0,2,1,3,NaN,Test,87,183.00,4,27.00


In [23]:
df_train.isna().sum()

quoted_amt           87
credit_score        224
policy_id             0
number_drivers        0
num_loaned_veh        0
num_owned_veh         0
num_leased_veh        0
total_number_veh      0
convert_ind           0
split                 0
driver_age            0
n_safty_rating       62
car_no                0
sum_age               0
dtype: int64

In [24]:
df_train = df_train.dropna(axis=0)

In [25]:
df_train = df_train.reset_index()

In [26]:
df_train['quoted_amt']

0         $5,153
1         $3,090
2        $14,917
3         $4,620
4        $11,470
          ...   
36494     $4,042
36495     $5,743
36496     $2,230
36497     $8,428
36498     $1,220
Name: quoted_amt, Length: 36499, dtype: object

In [27]:
list_quoted_price = []
for i in range(len(df_train)):
    list_quoted_price.append(int(re.sub(r'[^0-9]', '', df_train['quoted_amt'][i])))

In [28]:
df_train = df_train.drop(['quoted_amt'], axis = 'columns')
df_train['quoted_amt'] = list_quoted_price


In [29]:
df_train = df_train.drop(['index', 'policy_id', 'split'], axis = 'columns')

In [30]:
df_train

,credit_score,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,driver_age,n_safty_rating,car_no,sum_age,quoted_amt
0,613.0,2,1,2,0,3,0.0,99,148.0,4,21.0,5153
1,631.0,2,1,2,1,4,0.0,98,140.0,4,28.0,3090
2,602.0,2,0,1,2,3,0.0,113,163.0,4,18.0,14917
3,704.0,1,2,2,0,4,0.0,18,66.0,5,30.0,4620
4,611.0,4,2,2,2,6,0.0,93,297.0,3,21.0,11470
...,...,...,...,...,...,...,...,...,...,...,...,...
36494,590.0,1,2,2,0,4,0.0,52,90.0,4,20.0,4042
36495,688.0,2,2,2,0,4,0.0,138,147.0,4,25.0,5743
36496,637.0,1,1,3,1,5,0.0,19,39.0,2,14.0,2230
36497,581.0,2,2,2,0,4,1.0,106,156.0,5,24.0,8428


In [31]:
x_train = df_train.drop(['convert_ind'], axis='columns')
y_train = df_train['convert_ind']

In [32]:
x = np.asarray(x_train)
y = np.asarray(y_train)


x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [393]:
model = XGBClassifier(random_state=42, scale_pos_weight=8, eval_metric='auc')
# dists = { 'n_estimators' : [10, 100],
#            'max_depth' : [6, 8, 10],
#            'min_samples_leaf' : [8, 12],
#            'min_samples_split' : [8, 16]
#             }

# clf = RandomForestClassifier(random_state=42, class_weight='balanced')

# clf1 = RandomizedSearchCV(
#     clf,
#     param_distributions=dists,
#     n_iter = 500,
#     cv = 5, 
#     scoring = 'accuracy',
#     verbose = 1,
#     random_state = 42
# )

model.fit(x, y)

result = model.predict_proba(df_test)



In [394]:
model.predict(df_test)

array([0., 1., 0., ..., 1., 1., 1.])

In [391]:
list_prob = []
for i in range(len(result)):
    list_prob.append(result[i][1])

In [392]:
result

array([[0.6368787 , 0.36312127],
       [0.446369  , 0.553631  ],
       [0.6915606 , 0.30843934],
       ...,
       [0.48531437, 0.51468563],
       [0.46413517, 0.53586483],
       [0.458508  , 0.541492  ]], dtype=float32)

In [398]:
list_prob

[0.36312127,
 0.553631,
 0.30843934,
 0.6083458,
 0.47599486,
 0.5374269,
 0.54588383,
 0.43802652,
 0.56867105,
 0.71208745,
 0.55783916,
 0.50081915,
 0.48052415,
 0.6496163,
 0.28737503,
 0.4755384,
 0.50415033,
 0.5744132,
 0.44882542,
 0.41042113,
 0.5885632,
 0.53019005,
 0.62978756,
 0.20497732,
 0.50224113,
 0.6362943,
 0.27583492,
 0.6252817,
 0.2149445,
 0.46222952,
 0.21450593,
 0.20171036,
 0.5528529,
 0.7052145,
 0.45855054,
 0.49091017,
 0.62134516,
 0.53147316,
 0.36274594,
 0.25672334,
 0.5833522,
 0.20092422,
 0.47773153,
 0.48554906,
 0.33234692,
 0.5657636,
 0.57909805,
 0.5695424,
 0.48001102,
 0.5087035,
 0.49686685,
 0.68321776,
 0.39645886,
 0.51159245,
 0.45263052,
 0.50919414,
 0.5513547,
 0.58311296,
 0.5039967,
 0.31452197,
 0.5520298,
 0.35987368,
 0.51159716,
 0.64533925,
 0.35094175,
 0.46874207,
 0.60764277,
 0.38420248,
 0.58494586,
 0.6090456,
 0.49285007,
 0.21050468,
 0.161012,
 0.6223775,
 0.5918146,
 0.17641194,
 0.55718976,
 0.5788378,
 0.5554577,


In [410]:
list(df_test['policy_id'])

['policy_89288',
 'policy_23460',
 'policy_43809',
 'policy_4590',
 'policy_65525',
 'policy_86829',
 'policy_7184',
 'policy_79862',
 'policy_78687',
 'policy_46401',
 'policy_35767',
 'policy_16584',
 'policy_89554',
 'policy_49763',
 'policy_95522',
 'policy_290',
 'policy_10909',
 'policy_16637',
 'policy_47833',
 'policy_30989',
 'policy_19204',
 'policy_68894',
 'policy_28455',
 'policy_1395',
 'policy_14253',
 'policy_55154',
 'policy_96941',
 'policy_64239',
 'policy_86185',
 'policy_40115',
 'policy_79475',
 'policy_34094',
 'policy_36006',
 'policy_52858',
 'policy_92289',
 'policy_73025',
 'policy_40143',
 'policy_20864',
 'policy_25677',
 'policy_32945',
 'policy_26966',
 'policy_33524',
 'policy_67300',
 'policy_40375',
 'policy_69025',
 'policy_42416',
 'policy_82735',
 'policy_51793',
 'policy_53164',
 'policy_22836',
 'policy_69395',
 'policy_84485',
 'policy_93277',
 'policy_14132',
 'policy_55179',
 'policy_70433',
 'policy_77792',
 'policy_12110',
 'policy_12627',
 '

In [419]:
dict={'policy_id':list(df_test['policy_id']),
     'TARGET':list_prob}

In [420]:
final_csv = pd.DataFrame(dict)

In [424]:
final_csv.columns

Index(['policy_id', 'TARGET'], dtype='object')

In [425]:
final_csv.to_csv("final.csv")

In [388]:
result[0][1]

0.36312127

In [ ]:
# print('Best Hyperparameters: ', model.best_params_)
y_test_pred = model.predict(x_train_test)


pd.set_option('display.float_format',  '{:.2f}'.format)
results = pd.DataFrame(metrics.classification_report(y_train_test, y_test_pred, output_dict=True)).to_latex()
print(results)

In [372]:
y_test_pred

array([1., 1., 0., ..., 0., 0., 0.])

In [373]:
roc_auc_score(y_train_test, y_test_pred)

0.584667108052456

In [334]:
clf = RandomForestClassifier(random_state=42, class_weight='balanced')
clf.fit(x_train_train, y_train_train)
predicted = clf.predict(x_train_test)

In [341]:
list(y_train_test).count(0.0)

6488

In [344]:
list(y_train_test).count(1.0)

812

In [362]:
6488/812

7.990147783251231

In [323]:
temp = clf.predict_proba(x_train_test)

In [317]:
predicted = (temp [:,1] >= 0.3).astype('int')

In [328]:
accuracy = accuracy_score(y_train_test, predicted)

In [329]:
accuracy

0.8749315068493151

In [330]:
roc_auc_score(y_train_test, predicted)

0.5062225525866625

In [331]:
predicted

array([0, 0, 0, ..., 0, 0, 0])

In [332]:
pd.set_option('display.float_format',  '{:.2f}'.format)
results = pd.DataFrame(metrics.classification_report(y_train_test, predicted, output_dict=True)).to_latex()
print(results)

\begin{tabular}{lrrrrr}
\toprule
{} &     0.0 &    1.0 &  accuracy &  macro avg &  weighted avg \\
\midrule
precision &    0.89 &   0.17 &      0.87 &       0.53 &          0.81 \\
recall    &    0.98 &   0.03 &      0.87 &       0.51 &          0.87 \\
f1-score  &    0.93 &   0.05 &      0.87 &       0.49 &          0.84 \\
support   & 6488.00 & 812.00 &      0.87 &    7300.00 &       7300.00 \\
\bottomrule
\end{tabular}



/var/folders/1v/cyx67vkj1r3bwgnv9l4t23vh0000gn/T/ipykernel_26885/635306558.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results = pd.DataFrame(metrics.classification_report(y_train_test, predicted, output_dict=True)).to_latex()


In [250]:
y_test_pred

array([1., 0., 0., ..., 0., 0., 0.])

In [233]:
x_train_train

array([[6.550e+02, 2.000e+00, 2.000e+00, ..., 6.000e+00, 3.400e+01,
        4.616e+03],
       [5.770e+02, 1.000e+00, 2.000e+00, ..., 3.000e+00, 2.100e+01,
        7.447e+03],
       [6.040e+02, 2.000e+00, 1.000e+00, ..., 2.000e+00, 1.500e+01,
        5.407e+03],
       ...,
       [5.500e+02, 3.000e+00, 2.000e+00, ..., 3.000e+00, 1.400e+01,
        7.769e+03],
       [5.880e+02, 1.000e+00, 2.000e+00, ..., 2.000e+00, 1.100e+01,
        6.758e+03],
       [4.790e+02, 5.000e+00, 1.000e+00, ..., 2.000e+00, 9.000e+00,
        7.544e+03]])

In [247]:
y_train_train

array([0., 0., 0., ..., 0., 0., 0.])

In [244]:
clf.fit(x_train_train, y_train_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [236]:
from sklearn.metrics import accuracy_score

In [245]:
y_pred = clf.predict(x_train_test)

In [246]:
for i in y_pred:
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [231]:
list_one = []
list_zero = []

for i in y_train_test:
    if i==1.0:
        list_one.append(i)
    if i ==0.0:
        list_zero.append(i)

In [239]:
len(list_zero)/(len(list_zero)+len(list_one))

0.8887671232876713

In [211]:
accuracy_score(y_pred, y_train_test)

0.8887671232876713

In [40]:
import catboost
from catboost import CatBoostClassifier

clf_catboost = CatBoostClassifier(iterations=100, random_state=42)
clf_catboost.fit(x_train_train, y_train_train, eval_set=(x_train_test, y_train_test))

Learning rate set to 0.198543
0:	learn: 0.5854141	test: 0.5854268	best: 0.5854268 (0)	total: 67.3ms	remaining: 6.66s
1:	learn: 0.5141890	test: 0.5140569	best: 0.5140569 (1)	total: 69.4ms	remaining: 3.4s
2:	learn: 0.4653446	test: 0.4651484	best: 0.4651484 (2)	total: 71.9ms	remaining: 2.32s
3:	learn: 0.4305501	test: 0.4303389	best: 0.4303389 (3)	total: 73.8ms	remaining: 1.77s
4:	learn: 0.4057409	test: 0.4056365	best: 0.4056365 (4)	total: 75.7ms	remaining: 1.44s
5:	learn: 0.3881740	test: 0.3882024	best: 0.3882024 (5)	total: 78.3ms	remaining: 1.23s
6:	learn: 0.3754685	test: 0.3754999	best: 0.3754999 (6)	total: 80.3ms	remaining: 1.07s
7:	learn: 0.3659408	test: 0.3661273	best: 0.3661273 (7)	total: 82.9ms	remaining: 954ms
8:	learn: 0.3594617	test: 0.3597418	best: 0.3597418 (8)	total: 85.7ms	remaining: 866ms
9:	learn: 0.3542958	test: 0.3547044	best: 0.3547044 (9)	total: 88.2ms	remaining: 794ms
10:	learn: 0.3506180	test: 0.3511605	best: 0.3511605 (10)	total: 90.8ms	remaining: 735ms
11:	learn: 0

In [41]:
cb_auc = roc_auc_score(y_train_test, clf_catboost.predict_proba(x_train_test)[:,1])
print("AUC score of CatBoost: {:.3f}".format(cb_auc))

AUC score of CatBoost: 0.642


In [58]:
### how to deal with categorical data?
### hyperparameter setting?

In [146]:
df_test = df_test.reset_index()

In [147]:
df_test

,index,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
0,1,"$9,870",576.0,policy_89288,1,1,1,2,4,NaN,Test,56,62.0,4,11.0
1,3,"$2,860",573.0,policy_23460,1,2,3,0,5,NaN,Test,30,90.0,2,11.0
2,7,"$2,980",679.0,policy_43809,4,0,2,0,2,NaN,Test,91,238.0,2,6.0
3,8,"$1,945",773.0,policy_4590,3,1,3,2,6,NaN,Test,144,194.0,4,9.0
4,9,"$5,829",680.0,policy_65525,1,2,3,0,5,NaN,Test,55,72.0,5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,49137,"$2,922",727.0,policy_8331,1,3,3,0,6,NaN,Test,43,28.0,4,17.0
12287,49144,"$5,437",658.0,policy_4090,5,1,2,0,3,NaN,Test,133,318.0,4,19.0
12288,49146,"$3,287",586.0,policy_39956,1,2,2,0,4,NaN,Test,18,51.0,1,6.0
12289,49150,"$3,567",693.0,policy_68766,3,0,2,1,3,NaN,Test,87,183.0,4,27.0


In [148]:
df_test.isna().sum()

index                   0
quoted_amt             25
credit_score           76
policy_id               0
number_drivers          0
num_loaned_veh          0
num_owned_veh           0
num_leased_veh          0
total_number_veh        0
convert_ind         12291
split                   0
driver_age              0
n_safty_rating         15
car_no                  0
sum_age                 0
dtype: int64

In [149]:
df_test = df_test.fillna({'credit_score': df_test['credit_score'].mean()})

In [150]:
df_test = df_test.fillna({'quoted_amt': df_test['quoted_amt'].mode()[0]})

In [151]:
df_test = df_test.fillna({'n_safty_rating': df_test['n_safty_rating'].mean()})

In [152]:
df_test

,index,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
0,1,"$9,870",576.000000,policy_89288,1,1,1,2,4,NaN,Test,56,62.0,4,11.0
1,3,"$2,860",573.000000,policy_23460,1,2,3,0,5,NaN,Test,30,90.0,2,11.0
2,7,"$2,980",679.000000,policy_43809,4,0,2,0,2,NaN,Test,91,238.0,2,6.0
3,8,"$1,945",773.000000,policy_4590,3,1,3,2,6,NaN,Test,144,194.0,4,9.0
4,9,"$5,829",680.000000,policy_65525,1,2,3,0,5,NaN,Test,55,72.0,5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,49137,"$2,922",727.000000,policy_8331,1,3,3,0,6,NaN,Test,43,28.0,4,17.0
12287,49144,"$5,437",658.000000,policy_4090,5,1,2,0,3,NaN,Test,133,318.0,4,19.0
12288,49146,"$3,287",586.000000,policy_39956,1,2,2,0,4,NaN,Test,18,51.0,1,6.0
12289,49150,"$3,567",693.000000,policy_68766,3,0,2,1,3,NaN,Test,87,183.0,4,27.0


In [153]:
df_test.isna().sum()

index                   0
quoted_amt              0
credit_score            0
policy_id               0
number_drivers          0
num_loaned_veh          0
num_owned_veh           0
num_leased_veh          0
total_number_veh        0
convert_ind         12291
split                   0
driver_age              0
n_safty_rating          0
car_no                  0
sum_age                 0
dtype: int64

In [113]:
df_test

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,driver_age,n_safty_rating,car_no,sum_age
1,"$9,870",576.0,policy_89288,1,1,1,2,4,<bound method DataFrame.mode of quoted_a...,Test,56,62.0,4,11.0
3,"$2,860",573.0,policy_23460,1,2,3,0,5,<bound method DataFrame.mode of quoted_a...,Test,30,90.0,2,11.0
7,"$2,980",679.0,policy_43809,4,0,2,0,2,<bound method DataFrame.mode of quoted_a...,Test,91,238.0,2,6.0
8,"$1,945",773.0,policy_4590,3,1,3,2,6,<bound method DataFrame.mode of quoted_a...,Test,144,194.0,4,9.0
9,"$5,829",680.0,policy_65525,1,2,3,0,5,<bound method DataFrame.mode of quoted_a...,Test,55,72.0,5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49137,"$2,922",727.0,policy_8331,1,3,3,0,6,<bound method DataFrame.mode of quoted_a...,Test,43,28.0,4,17.0
49144,"$5,437",658.0,policy_4090,5,1,2,0,3,<bound method DataFrame.mode of quoted_a...,Test,133,318.0,4,19.0
49146,"$3,287",586.0,policy_39956,1,2,2,0,4,<bound method DataFrame.mode of quoted_a...,Test,18,51.0,1,6.0
49150,"$3,567",693.0,policy_68766,3,0,2,1,3,<bound method DataFrame.mode of quoted_a...,Test,87,183.0,4,27.0


In [154]:
df_test = df_test.drop(['convert_ind'], axis = 'columns')

In [155]:
df_test = df_test.drop(['index'], axis= 'columns')

In [156]:
df_test

,quoted_amt,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,split,driver_age,n_safty_rating,car_no,sum_age
0,"$9,870",576.000000,policy_89288,1,1,1,2,4,Test,56,62.0,4,11.0
1,"$2,860",573.000000,policy_23460,1,2,3,0,5,Test,30,90.0,2,11.0
2,"$2,980",679.000000,policy_43809,4,0,2,0,2,Test,91,238.0,2,6.0
3,"$1,945",773.000000,policy_4590,3,1,3,2,6,Test,144,194.0,4,9.0
4,"$5,829",680.000000,policy_65525,1,2,3,0,5,Test,55,72.0,5,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,"$2,922",727.000000,policy_8331,1,3,3,0,6,Test,43,28.0,4,17.0
12287,"$5,437",658.000000,policy_4090,5,1,2,0,3,Test,133,318.0,4,19.0
12288,"$3,287",586.000000,policy_39956,1,2,2,0,4,Test,18,51.0,1,6.0
12289,"$3,567",693.000000,policy_68766,3,0,2,1,3,Test,87,183.0,4,27.0


In [ ]:
df

In [157]:
list_quoted_price = []
for i in range(len(df_test)):
    list_quoted_price.append(int(re.sub(r'[^0-9]', '', df_test['quoted_amt'][i])))

In [158]:
df_test = df_test.drop(['quoted_amt'], axis = 'columns')
df_test['quoted_amt'] = list_quoted_price


In [159]:
df_test

,credit_score,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,split,driver_age,n_safty_rating,car_no,sum_age,quoted_amt
0,576.000000,policy_89288,1,1,1,2,4,Test,56,62.0,4,11.0,9870
1,573.000000,policy_23460,1,2,3,0,5,Test,30,90.0,2,11.0,2860
2,679.000000,policy_43809,4,0,2,0,2,Test,91,238.0,2,6.0,2980
3,773.000000,policy_4590,3,1,3,2,6,Test,144,194.0,4,9.0,1945
4,680.000000,policy_65525,1,2,3,0,5,Test,55,72.0,5,21.0,5829
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,727.000000,policy_8331,1,3,3,0,6,Test,43,28.0,4,17.0,2922
12287,658.000000,policy_4090,5,1,2,0,3,Test,133,318.0,4,19.0,5437
12288,586.000000,policy_39956,1,2,2,0,4,Test,18,51.0,1,6.0,3287
12289,693.000000,policy_68766,3,0,2,1,3,Test,87,183.0,4,27.0,3567


In [160]:
df_test = df_test.drop(['policy_id', 'split'], axis='columns')

In [161]:
df_test

,credit_score,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,driver_age,n_safty_rating,car_no,sum_age,quoted_amt
0,576.000000,1,1,1,2,4,56,62.0,4,11.0,9870
1,573.000000,1,2,3,0,5,30,90.0,2,11.0,2860
2,679.000000,4,0,2,0,2,91,238.0,2,6.0,2980
3,773.000000,3,1,3,2,6,144,194.0,4,9.0,1945
4,680.000000,1,2,3,0,5,55,72.0,5,21.0,5829
...,...,...,...,...,...,...,...,...,...,...,...
12286,727.000000,1,3,3,0,6,43,28.0,4,17.0,2922
12287,658.000000,5,1,2,0,3,133,318.0,4,19.0,5437
12288,586.000000,1,2,2,0,4,18,51.0,1,6.0,3287
12289,693.000000,3,0,2,1,3,87,183.0,4,27.0,3567


In [178]:
df_test.isna().sum()

credit_score        0
number_drivers      0
num_loaned_veh      0
num_owned_veh       0
num_leased_veh      0
total_number_veh    0
driver_age          0
n_safty_rating      0
car_no              0
sum_age             0
quoted_amt          0
dtype: int64

In [190]:
model = XGBClassifier(random_state=42)

array([0., 0., 0., ..., 0., 0., 0.])

In [199]:
x.shape

(36499, 11)

In [200]:
y.shape

(36499,)

In [213]:
model.fit(x, y)

XGBClassifier(random_state=42)

In [214]:
df_test = np.asarray(df_test)


In [215]:
df_test.shape

(12291, 11)

In [216]:
df_test

array([[5.76000000e+02, 1.00000000e+00, 1.00000000e+00, ...,
        4.00000000e+00, 1.10000000e+01, 9.87000000e+03],
       [5.73000000e+02, 1.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 1.10000000e+01, 2.86000000e+03],
       [6.79000000e+02, 4.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 6.00000000e+00, 2.98000000e+03],
       ...,
       [5.86000000e+02, 1.00000000e+00, 2.00000000e+00, ...,
        1.00000000e+00, 6.00000000e+00, 3.28700000e+03],
       [6.93000000e+02, 3.00000000e+00, 0.00000000e+00, ...,
        4.00000000e+00, 2.70000000e+01, 3.56700000e+03],
       [6.41135898e+02, 3.00000000e+00, 1.00000000e+00, ...,
        3.00000000e+00, 1.20000000e+01, 2.03600000e+03]])

In [220]:
y_pred = model.predict(df_test)

In [221]:
y_pred = np.asarray(y_pred)

In [223]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [222]:
import csv

f = open("data2.csv", "w")
writer = csv.writer(f)
 
writer.writerows(y_pred) ## 여기 주목!
f.close()

Error: iterable expected, not numpy.float64

In [470]:
df = pd.read_csv('./data_with_agent.csv')

In [471]:
df_test = df[df['split']=='Test']

In [472]:
df_train = df[df['split']=='Train']

In [473]:
df_test = df_test

In [474]:
df_train_target = df_train['convert_ind']

In [475]:
df_train_x = df_train.drop(['convert_ind', 'split', 'Unnamed: 0', 'X'], axis='columns')

In [476]:
df_test = df_test.drop(['convert_ind', 'split', 'Unnamed: 0', 'X'], axis='columns')

In [477]:
df_test

,discount,Home_policy_ind,df_prior_carrier,df_cov_package,df_cat_zone,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,driver_age,freq_agent
1,0,0,3,2,3,1,1,1,2,56,0
3,0,0,6,1,2,1,2,3,0,30,1
7,0,0,5,0,3,4,0,2,0,91,1
8,0,0,9,2,1,3,1,3,2,144,1
9,0,0,0,2,0,1,2,3,0,55,1
...,...,...,...,...,...,...,...,...,...,...,...
49137,1,1,3,2,4,1,3,3,0,43,0
49144,0,0,1,1,2,5,1,2,0,133,1
49146,0,0,3,2,3,1,2,2,0,18,1
49150,0,0,7,1,0,3,0,2,1,87,1


In [469]:
df_train_x

,discount,Home_policy_ind,df_prior_carrier,df_cov_package,df_cat_zone,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,driver_age,freq_agent
0,1,1,0,0,1,2,1,2,0,99,1
2,0,0,3,1,1,2,1,2,1,98,1
4,0,0,2,1,3,2,0,1,2,113,0
5,0,0,3,0,0,1,2,2,0,18,1
6,0,0,4,0,3,4,2,2,2,93,0
...,...,...,...,...,...,...,...,...,...,...,...
49157,1,1,8,0,3,1,2,2,0,52,1
49158,1,1,5,1,1,2,2,2,0,138,1
49159,0,0,6,2,1,1,1,3,1,19,1
49160,1,1,5,0,3,2,2,2,0,106,1


In [478]:
model = XGBClassifier(random_state=42, scale_pos_weight=8, eval_metric='auc')


df_train_x = np.array(df_train_x)

model.fit(df_train_x, df_train_target)




XGBClassifier(eval_metric='auc', random_state=42, scale_pos_weight=8)

In [479]:
df_test = np.array(df_test)

In [480]:
result = model.predict_proba(df_test)

In [482]:
result

array([[0.542053  , 0.45794705],
       [0.588447  , 0.41155306],
       [0.7588382 , 0.24116184],
       ...,
       [0.5895572 , 0.41044286],
       [0.45575684, 0.54424316],
       [0.5791973 , 0.4208027 ]], dtype=float32)

In [483]:
list_prob = []
for i in range(len(result)):
    list_prob.append(result[i][1])

In [492]:
len(list_prob)

12291

In [493]:
len(list(df_test['policy_id']))

12291

In [494]:
dict={'policy_id':list(df_test['policy_id']),
     'TARGET':list_prob}

In [496]:
final_csv = pd.DataFrame(dict)

In [497]:
final_csv.to_csv("final.csv")